In [37]:
import pandas as pd
PATH = 'epir_train/'
articles = pd.read_csv(PATH + 'articles.csv')
life_situations = pd.read_csv(PATH + 'life_situations.csv')
news = pd.read_csv(PATH + 'news.csv')
services = pd.read_csv(PATH + 'services.csv')

In [38]:
def clean_df(df):
    # rename Unnamed: 0 to true_id
    df = df.rename(columns={'Unnamed: 0': 'true_id'})
    
    text_cols = []
    for col in df.columns:
        if col not in ['id', 'sys_lang', 'subid', 'URL']:
            text_cols.append(col)

    df[text_cols] = df[text_cols].fillna('').astype(str)
    mask = df[text_cols].apply(lambda x: x.str.split().str.len() < 5).all(axis=1)

    print("Rows that will be deleted:")
    print(df[mask])

    df = df.drop(df[mask].index)
    return df

In [39]:
# apply cleaning for all dfs
articles = clean_df(articles)
life_situations = clean_df(life_situations)
news = clean_df(news)
services = clean_df(services)

In [ ]:
# list the column names for all dataframes
print("Articles columns:")
print(articles.columns)
print("Life situations columns:")
print(life_situations.columns)
print("News columns:")
print(news.columns)
print("Services columns:")
print(services.columns)

Articles columns:
Index(['true_id', 'id', 'sys_lang', 'projects', 'title', 'content', 'url'], dtype='object')
Life situations columns:
Index(['true_id', 'id', 'sys_lang', 'intro', 'title_main', 'title_sub',
       'subid', 'instruction', 'URL'],
      dtype='object')
News columns:
Index(['true_id', 'id', 'sys_lang', 'projects', 'title', 'body',
       'short_description', 'url'],
      dtype='object')
Services columns:
Index(['true_id', 'id', 'sys_lang', 'additional_info', 'description',
       'full_title', 'short_title', 'title', 'result_description', 'url'],
      dtype='object')


In [ ]:
# save the dataframes in data folder
articles.to_csv('data/cleaned/articles.csv')
news.to_csv('data/cleaned/news.csv')
life_situations.to_csv('data/cleaned/life_situations.csv')
services.to_csv('data/cleaned/services.csv')